This script aims to convert json file into csv file


In [8]:
import json
import os
import pandas as pd

### 1. Load data

In [ ]:
# Convert all JSON files in the input folder to CSVs in the processed folder.
current_dir = globals().get('current_dir', os.getcwd())
input_path = os.path.join(current_dir, '../data/mobidata/raw/json_files/')
output_path = os.path.join(current_dir, '../data/mobidata/raw/csv_files/')
os.makedirs(output_path, exist_ok=True)

In [ ]:

json_files = sorted([p for p in os.listdir(input_path) if p.lower().endswith('.json')])

for fname in json_files:
    input_file_path = os.path.join(input_path, fname)
    with open(input_file_path, 'r', encoding='utf-8') as f:
        js = json.load(f)

    # Flatten top-level metadata (except 'features') into scalar columns where possible
    top_meta = {}
    for k, v in js.items():
        if k == 'features':
            continue
        if isinstance(v, dict):
            # flatten one level for common nested dicts like crs: crs.properties.name -> crs_properties_name
            for k2, v2 in v.items():
                if isinstance(v2, dict):
                    for k3, v3 in v2.items():
                        top_meta[f"{k}_{k2}_{k3}"] = v3
                else:
                    top_meta[f"{k}_{k2}"] = v2
        else:
            top_meta[k] = v

    features = js.get('features')
    if features:
        rows = []
        for feat in features:
            row = {}
            # include file-level metadata for every feature row
            row.update(top_meta)

            # feature-level top values
            row['feature_type'] = feat.get('type')
            row['feature_id'] = feat.get('id')
            # optional geometry_name
            if 'geometry_name' in feat:
                row['geometry_name'] = feat.get('geometry_name')

            # flatten geometry: keep coordinates, also split x/y for points
            geom = feat.get('geometry', {}) or {}
            row['geometry_type'] = geom.get('type')
            coords = geom.get('coordinates')
            row['geometry_coordinates'] = coords

            # if it's a simple Point [x, y], expose x/y columns
            try:
                if isinstance(coords, (list, tuple)) and len(coords) >= 2 and all(isinstance(c, (int, float)) for c in coords[:2]):
                    row['geometry_x'] = coords[0]
                    row['geometry_y'] = coords[1]
            except Exception:
                pass

            # flatten properties (if dict)
            props = feat.get('properties', {}) or {}
            if isinstance(props, dict):
                for k, v in props.items():
                    # avoid overwriting existing keys (prefer property namespace if clash)
                    if k in row:
                        row[f"prop_{k}"] = v
                    else:
                        row[k] = v
            else:
                # if properties are not a dict, store as-is
                row['properties'] = props

            rows.append(row)

        df_out = pd.DataFrame(rows)
    else:
        # fallback: try to flatten top-level JSON
        df_out = pd.json_normalize(js)

    out_fname = os.path.splitext(fname)[0] + '.csv'
    out_path = os.path.join(output_path, out_fname)
    df_out.to_csv(out_path, index=False)
    print(f'Wrote {out_path} ({len(df_out)} rows)')

Wrote /Users/jiatong_liu/Documents/CorrelAid/lc-rheinmain-mobidta-task/src/../data/mobidata/raw/csv_files/baustellen_vorschau.csv (270 rows)
Wrote /Users/jiatong_liu/Documents/CorrelAid/lc-rheinmain-mobidta-task/src/../data/mobidata/raw/csv_files/dynamische_tafeln.csv (3 rows)


### 2. Check json files

In [ ]:
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [11]:
# High-level info
print(type(data))
print("Top-level keys:", list(data.keys()))
print("totalFeatures:", data.get("totalFeatures"))
print("numberReturned:", data.get("numberReturned"))
print("timeStamp:", data.get("timeStamp"))

<class 'dict'>
Top-level keys: ['type', 'features', 'totalFeatures', 'numberMatched', 'numberReturned', 'timeStamp', 'crs']
totalFeatures: 270
numberReturned: 270
timeStamp: 2025-10-06T14:15:33.080Z


In [12]:
# Features summary and keys
features = data.get("features", [])
print(f"Number of features: {len(features)}")
if features:
    f0 = features[0]
    print("Feature[0] keys:", list(f0.keys()))
    print("geometry keys:", list(f0.get("geometry", {}).keys()))
    print("properties keys (sample):", list(f0.get("properties", {}).keys()))

Number of features: 270
Feature[0] keys: ['type', 'id', 'geometry', 'geometry_name', 'properties']
geometry keys: ['type', 'coordinates']
properties keys (sample): ['id', 'gemeinde', 'vorgangszeitraum_von', 'vorgangszeitraum_bis', 'art', 'lage', 'tagesbaustelle', 'verursacher', 'zusatzinfo', 'sperrung', 'projektnummer', 'vorgangsnummer', 'datenquelle', 'stand']


In [13]:
# Pretty-print the first feature (readable JSON)
if features:
    print(json.dumps(features[0], ensure_ascii=False, indent=2))

{
  "type": "Feature",
  "id": "baustellen_vorschau.1112",
  "geometry": {
    "type": "Point",
    "coordinates": [
      389426.38255268,
      5270752.73090236
    ]
  },
  "geometry_name": "geom",
  "properties": {
    "id": 1112,
    "gemeinde": null,
    "vorgangszeitraum_von": "2025-10-09T22:00:00Z",
    "vorgangszeitraum_bis": "2025-10-12T18:00:00Z",
    "art": "AK5_encours",
    "lage": "D105 Hésingue/St-Louis",
    "tagesbaustelle": null,
    "verursacher": null,
    "zusatzinfo": "Dévoiement de la chaussée et changements des limitations de vitesses - voir mesures d'exploitations ci-joints.",
    "sperrung": null,
    "projektnummer": "6769",
    "vorgangsnummer": null,
    "datenquelle": "Collectivité européenne d’Alsace",
    "stand": "2025-10-05T22:00:00Z"
  }
}


In [14]:
# Optional: tabular view of properties for the first N features
N = 10
props = [feat.get("properties", {}) for feat in features[:N]]
df = pd.json_normalize(props)
df.head()

,id,gemeinde,vorgangszeitraum_von,vorgangszeitraum_bis,art,lage,tagesbaustelle,verursacher,zusatzinfo,sperrung,projektnummer,vorgangsnummer,datenquelle,stand
0,1112,None,2025-10-09T22:00:00Z,2025-10-12T18:00:00Z,AK5_encours,D105 Hésingue/St-Louis,None,None,Dévoiement de la chaussée et changements des l...,None,6769,None,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
1,1117,None,2025-12-09T23:00:00Z,2025-10-12T10:00:00Z,CE3a Information,"D130 - Bœrsch, Grendelbruch, Le Hohwald, Natzw...",None,None,Manifestation sportive intitulée “Biathlon <br...,None,6795,None,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
2,1141,None,2025-10-19T22:00:00Z,2025-10-24T06:00:00Z,KC1_route-barree,D426 - Entre Obernai et Ottrott,None,None,Route barrée avec mise en place d'une déviatio...,None,6810,None,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
3,1142,None,2025-10-19T22:00:00Z,2025-11-21T17:00:00Z,Travaux,D426 - Gerstheim,None,None,Veuillez vous référer aux documents joints pou...,None,6805,None,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
4,1148,None,2025-10-12T22:00:00Z,2025-10-14T16:00:00Z,KC1_route-barree,D505 Soultz-Haut-Rhin,None,None,Travaux de voirie;<br />Veuillez consulter les...,None,6813,None,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z


### 3. Convert to CSV

In [15]:
df = pd.json_normalize(data)

In [16]:
df

,type,features,totalFeatures,numberMatched,numberReturned,timeStamp,crs.type,crs.properties.name
0,FeatureCollection,"[{'type': 'Feature', 'id': 'baustellen_vorscha...",270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832


In [18]:
output_path = os.path.join(current_dir, '../data/mobidata/processed/')  # Adjust path to the processed data folder
output_file = 'baustellen_vorschau.csv'
output_file_path = os.path.join(output_path, output_file)
df.to_csv(output_file_path, index=False)